In [1]:
import json
import re
data = []
with open("./Stat628/Total.json", encoding = "utf-8" ) as f:
     for line in f:
        data.append(json.loads(line))

In [2]:
catagory = []
for i in data:
    catagory.append(i["categories"])

In [3]:
#new is the index of Chinese Restautants
new = []
for i in catagory:
    if i == None:
        continue
    if "Chinese" in i:
        if ("Restaurant" in i) or ("Food" in i):
            new.append(catagory.index(i))

In [4]:
Chinese = [data[i] for i in new]

In [5]:
text = []
for i in Chinese:
    reviews = i["Review"]
    for n in reviews:
        text.append(n["text"])

In [6]:
from nltk import sent_tokenize

sentences = []
for review in text:
    sentences += sent_tokenize(review)
    
print(sentences[0])

My girlfriend and I went for dinner at Emerald Chinese after a Thursday night workout.


In [7]:
sentences = [re.split(r"\W+", sentence) for sentence in sentences]

In [8]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stop_words.discard("not")
stop_words.discard("no")
stop_words.discard("nor")

In [9]:
#delete the space and the stop words
sentences_clean = []
for sentence in sentences:
    cleaned = []
    for word in sentence:
        if (word != "") and (not word in stop_words):
            cleaned.append(word.lower())
    sentences_clean.append(cleaned)
            

In [10]:
sentences_clean[0]

['my',
 'girlfriend',
 'i',
 'went',
 'dinner',
 'emerald',
 'chinese',
 'thursday',
 'night',
 'workout']

In [11]:
from nltk import pos_tag
import nltk
from nltk.corpus import sentiwordnet as swn

def getsentiscore(sentence):
    words_tag = pos_tag(sentence)
    wnl = nltk.WordNetLemmatizer()

    score_list=[]

    tag_not_know = []
    emotion_not_know = []
    for idx2,t in enumerate(words_tag):

        newtag=''

        lemmatized=wnl.lemmatize(t[0])
        
        if t[1].startswith('NN'):

            newtag='n'

        elif t[1].startswith('JJ'):

            newtag='a'

        elif t[1].startswith('V'):

            newtag='v'

        elif t[1].startswith('R'):

            newtag='r'

        else:

            newtag='' 
            score_list.append(0)
            tag_not_know.append(t[0])

        if(newtag!=''):    

            synsets = list(swn.senti_synsets(lemmatized, newtag))

            #Getting average of all possible sentiments, as you requested        

            score=0

            if(len(synsets)>0):

                for syn in synsets:

                    score+=syn.pos_score()-syn.neg_score()

                score_list.append(score/len(synsets))
            else:
                score_list.append(0)
                emotion_not_know.append(t[0])
    
    result = sum(score_list)/len(score_list)
                
    return result;




In [16]:
#24 2-grams phrases
phrases_2 = [('dim','sum'), ('fried','rice'), ('orange', 'chicken'), ('chow', 'mein'), ('pad', 'thai'), ('sour', 'soup'), ('hot', 'pot'), ('spring', 'rolls'), ('lo', 'mein'), ('bbq', 'pork'), ('mongolian', 'beef'), ('egg', 'roll'), ('wonton', 'soup'), ('soy', 'sauce'), ('ice', 'cream'), ('pork', 'belly'), ('milk', 'tea'), ('fried', 'chicken'),  ('sour', 'chicken'), ('white', 'rice'), ('sesame', 'chicken'), ('chicken', 'wings'), ('peking', 'duck'), ('bubble', 'tea')]

In [17]:
#10 -grams phrases
phrases_3 = [('egg', 'drop', 'soup'), ('beef', 'noodle', 'soup'), ('kung', 'pao', 'chicken'), ('chicken', 'fried', 'rice'), ('pork', 'fried', 'rice'), ('black', 'bean', 'sauce'), ('shrimp', 'fried', 'rice'), ('xiao', 'long', 'bao'), ('egg', 'foo', 'young'), ('dan', 'dan', 'noodles')]

In [20]:
phrases_score = []
for phrase in phrases_2:
    score = []
    for sentence in sentences_clean:
        index = 0
        sentence_score = 0
        if phrase[0] in sentence:
            index = sentence.index(phrase[0])
            index_need = index + 1
            
            #ensure the phrase[0] not the last word of sentence
            if index_need < len(sentence):
                if sentence[index_need] == phrase[1]:
                    sentence_score = getsentiscore(sentence)
                    score.append(sentence_score)
                
    phrases_score.append(score)

In [21]:
phrases_2_score = []
for i in phrases_score:
    score = sum(i)/len(i)
    phrases_2_score.append(score)

In [28]:
phrases_2_score

[0.019719092884475077,
 -5.043413935204559e-05,
 0.005710735101401866,
 0.020303412384346468,
 0.013057871367722964,
 -0.02747252199597914,
 0.009738746287431186,
 0.012401644728690014,
 -0.007481219159794992,
 0.021802851582992926,
 -0.008561537373841193,
 0.00012875326863170605,
 0.01058066672162855,
 0.007341816131693261,
 0.013454453279596656,
 0.015601353462746284,
 0.0188111275592233,
 -0.01765283659580652,
 -0.010986022022170592,
 -0.005734049523914381,
 0.02389270223424345,
 -0.007963539008378463,
 0.01066225336761093,
 0.016622292236968764]

In [23]:
phrases_score1 = []
for phrase in phrases_3:
    score = []
    for sentence in sentences_clean:
        index = 0
        sentence_score = 0
        if phrase[0] in sentence:
            index = sentence.index(phrase[0])
            index_need = index + 1
            
            #ensure the phrase[0] not the last word of sentence
            if index_need < len(sentence):
                if sentence[index_need] == phrase[1]:
                    sentence_score = getsentiscore(sentence)
                    score.append(sentence_score)
                
    phrases_score1.append(score)

In [24]:
phrases_3_score = []
for i in phrases_score1:
    score = sum(i)/len(i)
    phrases_3_score.append(score)

In [26]:
score = phrases_2_score + phrases_3_score 
name = phrases_2 +phrases_3

In [48]:
import numpy as np
index = np.argsort(score)
name_sorted = [name[i] for i in index]
name_sorted.reverse() # decrease

In [49]:
name_sorted

[('sesame', 'chicken'),
 ('bbq', 'pork'),
 ('chow', 'mein'),
 ('dim', 'sum'),
 ('egg', 'drop', 'soup'),
 ('milk', 'tea'),
 ('bubble', 'tea'),
 ('pork', 'belly'),
 ('ice', 'cream'),
 ('pad', 'thai'),
 ('spring', 'rolls'),
 ('kung', 'pao', 'chicken'),
 ('dan', 'dan', 'noodles'),
 ('xiao', 'long', 'bao'),
 ('peking', 'duck'),
 ('wonton', 'soup'),
 ('hot', 'pot'),
 ('soy', 'sauce'),
 ('orange', 'chicken'),
 ('beef', 'noodle', 'soup'),
 ('egg', 'roll'),
 ('fried', 'rice'),
 ('white', 'rice'),
 ('lo', 'mein'),
 ('chicken', 'wings'),
 ('mongolian', 'beef'),
 ('sour', 'chicken'),
 ('egg', 'foo', 'young'),
 ('pork', 'fried', 'rice'),
 ('fried', 'chicken'),
 ('chicken', 'fried', 'rice'),
 ('shrimp', 'fried', 'rice'),
 ('black', 'bean', 'sauce'),
 ('sour', 'soup')]

In [56]:
import scipy
import numpy as np

In [57]:
phrases_score = np.array(phrases_score)
scipy.stats.ttest_ind(phrases_score[0],phrases_score[1])

Ttest_indResult(statistic=30.616121336708883, pvalue=2.597236590014156e-203)